## Import libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from os import path
import sklearn
%matplotlib qt

## Variables to set

In [5]:
DATASET = "capture_test_25samples.csv"
DATA_PATH = path.join("data", DATASET)
SAMPLES = 5

## Load dataset

In [6]:
dataset = pd.read_csv(DATA_PATH)
dataset

,AcX0,AcX1,AcX2,AcX3,AcX4,AcX5,AcX6,AcX7,AcX8,AcX9,...,Tmp90,Tmp91,Tmp92,Tmp93,Tmp94,Tmp95,Tmp96,Tmp97,Tmp98,Tmp99
0,-350,1726,-2916,1070,-1958,2404,1934,-3746,3062,4426,...,-2256,-2272,-2240,-2272,-2208,-2240,-2240,-2240,-2240,-2240
1,3278,620,-2004,-86,-1202,4588,-4540,3332,102,-1574,...,-2256,-2192,-2256,-2224,-2224,-2240,-2240,-2240,-2240,-2272
2,-2284,1962,1780,-3116,2620,4740,-1812,2844,-1148,76,...,-2256,-2224,-2256,-2240,-2256,-2256,-2224,-2288,-2256,-2224
3,1018,4048,-3362,2676,-2492,844,3218,564,-3708,4578,...,-2256,-2256,-2256,-2224,-2256,-2256,-2240,-2240,-2240,-2240
4,3476,536,-2012,2090,-1420,4566,-4352,-1826,-20,-1308,...,-2240,-2240,-2256,-2256,-2256,-2224,-2256,-2256,-2224,-2224
5,-2044,3318,1336,-3200,2760,2362,-2648,844,-3540,2058,...,-2208,-2272,-2240,-2256,-2272,-2240,-2224,-2256,-2256,-2256
6,1100,4044,-6160,3758,-2320,750,1992,-1698,-3538,4524,...,-2256,-2224,-2256,-2256,-2240,-2240,-2240,-2272,-2240,-2240
7,-1506,4544,-1924,2810,-768,526,2980,-5490,3574,-2422,...,-2224,-2224,-2256,-2224,-2256,-2256,-2256,-2224,-2288,-2256
8,3912,-2074,2338,2194,-1642,610,4128,-1154,882,-1842,...,-2272,-2208,-2240,-2240,-2240,-2256,-2256,-2256,-2256,-2256
9,2052,-1378,452,1280,-1808,-62,-1956,3026,2048,-4070,...,-2272,-2240,-2240,-2272,-2272,-2272,-2240,-2240,-2240,-2240


In [7]:
X = [i for i in range(100)]
print(X)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


## Generate a figure with plots of the initial samples

In [11]:
fig, axs = plt.subplots(SAMPLES,7, sharex='col', figsize=(21,SAMPLES*3))
for i in range(SAMPLES):
    for j in range(7):
        y = dataset.iloc[i,j*100:(j+1)*100]
        axs[i,j].plot(X, y)
        if i == 0:
            axs[i,j].set_title(dataset.columns[j*100][:-1], fontsize=15)

fig.suptitle("Exemplo das "+str(SAMPLES)+" primeiras amostras", fontsize=25)            
plt.show()
fig.savefig(path.join("..","results","plot",DATASET[:-4]+".PNG"), dpi=300)